In [1]:
import pandas as pd
import numpy as np
import re
import pickle
import math

In [4]:
def retr_model_info():
    with open('../model_and_cols.pkl', 'rb') as f:
        county_info_df = pickle.load(f)
        model = pickle.load(f)
        cols = pickle.load(f)
    return(county_info_df, model, cols)



def predict_bars(idx, county_info_df, model, cols, percentile=95):
    X = np.asarray(county_info_df.iloc[idx][cols]).reshape(1,-1)
    y = county_info_df.iloc[idx]['bars']
    y_pred = model.predict(X)
    return(max(0,int(y_pred)), int(y))

In [5]:
county_info_df, model, cols = retr_model_info()
r=[]
b=[]
for idx in range(len(county_info_df)):  
    pred_y, actual_y = predict_bars(idx, county_info_df, model, cols)
    r.append(round(pred_y/actual_y,2))
    b.append(round(pred_y,0))

In [6]:
r_series = pd.Series(r)
y_pred_series = pd.Series(b)
county_info_df['model'] = y_pred_series
county_info_df['model/actual'] = r_series
pd.set_option('display.max_rows', 500)
county_info_df.sort_values('bars', ascending=False)

,geo_id,state_name,state_code,county_name,area_sqmi,pop_est_2015,pop_den_2015,hotels,liquor_stores,household_median_income,bars,model,model/actual
610,17031,Illinois,IL,"Cook County, IL",945.333,5238216,5541.133125,982.0,616.0,55251,1460.0,1176,0.81
2623,48201,Texas,TX,"Harris County, TX",1704.855,4538028,2661.826372,1117.0,287.0,54457,1204.0,1026,0.85
204,6037,California,CA,"Los Angeles County, CA",4058.101,10170292,2506.170251,2424.0,1172.0,56196,1184.0,1167,0.99
1858,36061,New York,NY,"New York County, NY",22.656,1644518,72586.423023,863.0,378.0,72871,851.0,740,0.87
2294,42101,Pennsylvania,PA,"Philadelphia County, PA",134.178,1567442,11681.810729,169.0,187.0,38253,665.0,617,0.93
1851,36047,New York,NY,"Kings County, NY",69.813,2636735,37768.538811,386.0,393.0,48201,621.0,539,0.87
1748,32003,Nevada,NV,"Clark County, NV",7891.478,2114801,267.985414,530.0,83.0,51575,587.0,310,0.53
1868,36081,New York,NY,"Queens County, NY",108.768,2339150,21505.865696,363.0,342.0,57720,567.0,479,0.84
2537,48029,Texas,TX,"Bexar County, TX",1239.876,1897753,1530.599028,568.0,130.0,51150,542.0,485,0.89
2245,42003,Pennsylvania,PA,"Allegheny County, PA",730.077,1230459,1685.382501,206.0,198.0,53040,541.0,406,0.75


In [7]:
county_info_df.sort_values('model/actual', ascending=False).head(10)

,geo_id,state_name,state_code,county_name,area_sqmi,pop_est_2015,pop_den_2015,hotels,liquor_stores,household_median_income,bars,model,model/actual
1967,37155,North Carolina,NC,"Robeson County, NC",949.010,134197,141.407361,34.0,7.0,30608,2.0,32,16.0
2919,51550,Virginia,VA,"Chesapeake city, VA",340.800,235429,690.812793,60.0,10.0,68620,2.0,30,15.0
173,5125,Arkansas,AR,"Saline County, AR",723.619,117460,162.322990,27.0,3.0,55817,2.0,25,12.5
1770,33013,New Hampshire,NH,"Merrimack County, NH",934.134,147994,158.429090,55.0,9.0,65983,2.0,25,12.5
1768,33009,New Hampshire,NH,"Grafton County, NH",1708.860,89320,52.268764,128.0,12.0,55762,2.0,25,12.5
2821,51003,Virginia,VA,"Albemarle County, VA",720.802,105703,146.646374,44.0,8.0,68449,2.0,25,12.5
443,13115,Georgia,GA,"Floyd County, GA",509.923,96504,189.252103,32.0,12.0,41549,2.0,22,11.0
2861,51085,Virginia,VA,"Hanover County, VA",468.567,103227,220.303607,25.0,5.0,78645,2.0,21,10.5
2779,49005,Utah,UT,"Cache County, UT",1164.723,120783,103.701052,23.0,2.0,50497,2.0,21,10.5
2853,51069,Virginia,VA,"Frederick County, VA",413.499,83199,201.207258,34.0,6.0,69098,2.0,18,9.0


In [8]:
county_info_df['pop_est_2015'].median()

25698.5

In [9]:
best_to_open_df = county_info_df[['model/actual', 'county_name', 'area_sqmi', 'pop_est_2015']].sort_values('model/actual')
best_to_open_df.index = np.arange(1, len(best_to_open_df) + 1)
best_to_open_df.columns = ['model/actual', 'County and State', 'County Area', 'Population']
best_to_open_df.head(20)


,model/actual,County and State,County Area,Population
1,0.13,"Langlade County, WI",870.632,19223
2,0.15,"Juneau County, WI",767.044,26224
3,0.15,"Forest County, WI",1014.235,9057
4,0.15,"Lincoln County, WI",878.860,27980
5,0.15,"Taylor County, WI",974.890,20455
6,0.15,"Ashland County, WI",1045.037,15843
7,0.16,"Silver Bow County, MT",717.995,34622
8,0.16,"Rusk County, WI",913.600,14124
9,0.17,"Lafayette County, WI",633.588,16829
10,0.18,"Washburn County, WI",797.113,15552


In [10]:
worse_to_open_df = county_info_df[['model/actual', 'county_name', 'area_sqmi', 'pop_est_2015']].sort_values('model/actual', ascending=False)
worse_to_open_df.index = np.arange(1, len(best_to_open_df) + 1)
worse_to_open_df.columns = ['model/actual', 'County and State', 'County Area', 'Population']
worse_to_open_df.head(20)

,model/actual,County and State,County Area,Population
1,16.00,"Robeson County, NC",949.010,134197
2,15.00,"Chesapeake city, VA",340.800,235429
3,12.50,"Saline County, AR",723.619,117460
4,12.50,"Merrimack County, NH",934.134,147994
5,12.50,"Grafton County, NH",1708.860,89320
6,12.50,"Albemarle County, VA",720.802,105703
7,11.00,"Floyd County, GA",509.923,96504
8,10.50,"Hanover County, VA",468.567,103227
9,10.50,"Cache County, UT",1164.723,120783
10,9.00,"Frederick County, VA",413.499,83199
